# Disturbance & Event Guides: FaultAgent

This notebook provides a guide to the `FaultAgent`. This is a powerful agent used to simulate system failures, such as sensor malfunctions or actuator wear. Using the `FaultAgent` is critical for testing the **robustness and resilience** of your control strategies.

## 1. How It Works

The `FaultAgent` is initialized with a `fault_sequence`, which is a list of fault events. Each event is a dictionary that specifies:
- `type`: The type of fault (e.g., `'SensorDrift'`).
- `start_time`: The simulation time at which the fault begins.
- Other parameters specific to the fault type (e.g., `drift_rate`).

During the simulation, the `FaultAgent` activates these faults at their specified start times. The `output` of the agent represents the magnitude of the fault, which you can then apply to your system. For example, you could add the output of a `SensorDrift` fault to a real sensor's reading before it's sent to a controller.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.disturbances.agents import FaultAgent
from water_system_sdk.src.chs_sdk.modules.control.pid_controller import PIDController
from water_system_sdk.src.chs_sdk.modules.modeling.storage_models import LinearTank

## 2. Code Example: Simulating a Sensor Drift Fault

We will set up a PID control loop to maintain a tank's water level. Halfway through the simulation, we will use a `FaultAgent` to introduce a **sensor drift**, causing the sensor to report a slowly increasing incorrect value. We will then observe how this fault affects the PID controller's behavior.

In [ ]:
# 1. Define the fault sequence
fault_sequence = [
    {
        "type": "SensorDrift",
        "start_time": 100, # Fault starts at t=100s
        "drift_rate": 0.05 # The sensor reading will drift upwards by 0.05m every second
    }
]

# 2. Initialize the models
fault_agent = FaultAgent(fault_sequence=fault_sequence)
tank = LinearTank(area=1000.0, initial_level=10.0)
pid = PIDController(Kp=2.0, Ki=0.1, Kd=0.5, set_point=10.0)

# 3. Simulation Setup
dt = 1.0
n_steps = 200
history = {"time": [], "true_level": [], "sensed_level": [], "inflow": []}

# 4. Simulation Loop
for t in range(n_steps):
    # Get the fault signal
    fault_agent.step(t=t, dt=dt)
    fault_signal = fault_agent.output
    
    # The controller's feedback is the true level plus the fault signal
    true_level = tank.level
    sensed_level = true_level + fault_signal
    
    # PID calculates the required inflow based on the (incorrect) sensed level
    control_output = pid.step(dt=dt, feedback_signal=sensed_level)
    
    # Apply control to the tank
    tank.input.inflow = max(0, control_output)
    tank.step(dt=dt)
    
    # Record results
    history["time"].append(t)
    history["true_level"].append(true_level)
    history["sensed_level"].append(sensed_level)
    history["inflow"].append(tank.input.inflow)

print("Simulation complete.")

## 3. Visualization

Plotting the true water level vs. the sensed water level clearly shows the impact of the fault.

In [ ]:
plt.figure(figsize=(12, 6))

plt.plot(history['time'], history['true_level'], label='True Water Level', color='blue', linewidth=2)
plt.plot(history['time'], history['sensed_level'], label='Sensed Water Level (with fault)', color='orange', linestyle='--')
plt.axhline(y=10.0, color='r', linestyle=':', label='Setpoint (10.0m)')
plt.axvline(x=100, color='k', linestyle='--', label='Fault Start (t=100s)')

plt.title('Effect of Sensor Drift Fault on PID Control')
plt.xlabel('Time (s)')
plt.ylabel('Water Level (m)')
plt.legend()
plt.grid(True)
plt.show()

### Analysis of Results

The plot shows that for the first 100 seconds, the system is stable, and the true level matches the sensed level at the setpoint. 

At t=100s, the fault begins. The `sensed_level` (orange dashed line) starts to drift upwards. The PID controller, seeing this artificially high reading, thinks the water level is too high and starts to reduce the inflow to correct it. As a result, the `true_level` (blue line) begins to drop, moving away from the setpoint.

This example demonstrates how the `FaultAgent` can be used to create powerful test cases to evaluate how a control system behaves under non-ideal, real-world conditions.